# Gene Expression ODE Simulation
This notebook demonstrates how to simulate a simple gene expression model with various promoter strengths. We use:
- **Numpy** for numerical arrays
- **Pandas** for data manipulation
- **Scipy** (odeint) for solving the ODE system
- **Matplotlib** for visualization

We'll follow these steps:
1. Install & import required libraries
2. Define promoter strengths and parameters
3. Define the ODE system
4. Solve the ODE for each promoter
5. Transform & plot the results

You can execute each section cell-by-cell in Google Colab or Jupyter.

## 1. Install & Import Dependencies

*Uncomment the pip install commands if any library is missing in your environment.*

In [ ]:
# (Optional) Install packages if you need them.
# !pip install numpy pandas matplotlib scipy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint

print("All libraries imported successfully!")

## 2. Define Promoter Strengths & Parameters

Here we specify:
- **`k1_values`**: different promoter strengths.
- **`params`**: all other parameters (`d1`, `k2`, `d2`).
- **`initial_state`**: initial concentrations for `[gene, mRNA, protein]`.

In [ ]:
# 1) Define the promoter strengths (k1_values)
k1_values = {
    "J23100": 1.00,
    "J23101": 0.70,
    "J23106": 0.47,
    "J23116": 0.16,
    "J23117": 0.06
}

# 2) Define the additional parameters
#    d1: mRNA degradation rate, ~ half-life of 3 min => log(2)/3
#    k2: translation rate
#    d2: protein degradation rate
#    k1 will be assigned in the loop below.
params = {
    "k1": None,         # placeholder to be set per promoter
    "d1": np.log(2)/3,  # ~0.231, for 3-min half-life
    "k2": 8.23,         # translation rate
    "d2": 0.02          # protein degradation rate
}

# 3) Define the initial state [gene, mRNA, protein]
initial_state = [17, 0, 0]

print("Promoter strengths and parameters defined.")
print("k1_values:", k1_values)
print("params:", params)
print("Initial state (gene, mRNA, protein):", initial_state)

## 3. Define the ODE System

We define a function that returns the derivatives:
- Gene is constant (no creation/degradation in this model).
- \(\frac{d(mRNA)}{dt} = k_1 \cdot gene - d_1 \cdot mRNA\)
- \(\frac{d(protein)}{dt} = k_2 \cdot mRNA - d_2 \cdot protein\)

In [ ]:
def derivative(state, t, params):
    """
    Compute the derivatives for the gene expression system:
    dGene/dt = 0
    d(mRNA)/dt = k1 * gene - d1 * mRNA
    d(protein)/dt = k2 * mRNA - d2 * protein

    Parameters
    ----------
    state : array-like
        Current values of [gene, mRNA, protein].
    t : float
        Current time (not explicitly used here, but needed for odeint signature).
    params : dict
        Dictionary of parameters with keys: "k1", "d1", "k2", "d2".

    Returns
    -------
    list
        Derivatives [dGene, dmRNA, dProtein].
    """
    gene, mRNA, protein = state
    k1 = params["k1"]
    d1 = params["d1"]
    k2 = params["k2"]
    d2 = params["d2"]

    # Gene is assumed constant
    dGene = 0

    # Transcription minus mRNA degradation
    dmRNA = k1 * gene - d1 * mRNA

    # Translation minus protein degradation
    dProtein = k2 * mRNA - d2 * protein

    return [dGene, dmRNA, dProtein]

print("ODE derivative function defined.")

## 4. Solve the ODE for Each Promoter

We'll:
1. Create a time array from 0 to 180 minutes (step 0.1).
2. Loop through each promoter strength, set `params["k1"]`, solve, and store results.
3. Collect all solutions in a single DataFrame.

In [ ]:
# 1) Define time points in minutes.
times = np.arange(0, 180, 0.1)

# 2) Prepare a DataFrame to store results across promoters.
all_results = pd.DataFrame()

# 3) Loop over each promoter and solve.
for promoter_name, k1_val in k1_values.items():
    # Update k1
    params["k1"] = k1_val

    # Solve using odeint
    solution = odeint(derivative, initial_state, times, args=(params,))
    # solution shape: (len(times), 3) => columns = [gene, mRNA, protein]

    # Convert to a DataFrame for easy concatenation
    temp_df = pd.DataFrame({
        "time": times,
        "gene": solution[:, 0],
        "mRNA": solution[:, 1],
        "protein": solution[:, 2],
        "promoter": promoter_name
    })

    # Append to the master all_results DataFrame
    all_results = pd.concat([all_results, temp_df], ignore_index=True)

print("ODE solved for all promoters.")
print("all_results shape:", all_results.shape)
all_results.head()

## 5. Transform & Plot the Results

1. Melt the DataFrame from “wide” form (gene/mRNA/protein in separate columns) to “long” form (one column `component`, one column `value`).
2. Filter out zero or negative values (since we'll use a log scale).
3. Plot a single figure with lines for each component, using different styles for each promoter.
4. Switch the y-axis to a log scale.

In [ ]:
# 1) Melt from wide to long format.
melted = all_results.melt(
    id_vars=["time", "promoter"],
    value_vars=["gene", "mRNA", "protein"],
    var_name="component",
    value_name="value"
)

# 2) Filter out non-positive values (log scale doesn't handle <= 0)
melted = melted[melted["value"] > 0]

# 3) Create figure
plt.figure(figsize=(8, 6))

# Distinguish promoters by line style
line_styles = ["-", "--", "-.", ":", (0, (3,1,1,1))]
promoters = melted["promoter"].unique()
style_map = dict(zip(promoters, line_styles))

# Distinguish components by color
colors = {"gene": "blue", "mRNA": "green", "protein": "red"}
components = melted["component"].unique()

# 4) Plot each combination of component & promoter
for comp in components:
    for prom in promoters:
        subset = melted[(melted["component"] == comp) & (melted["promoter"] == prom)]
        plt.plot(
            subset["time"],
            subset["value"],
            label=f"{comp}, {prom}",
            linestyle=style_map[prom],
            color=colors[comp]
        )

# 5) Switch y-axis to log scale
plt.yscale("log")

# 6) Label axes and add a title
plt.xlabel("Time (min)")
plt.ylabel("Concentration (log scale)")
plt.title("Gene Expression Over Time for Different Promoter Strengths")

# 7) Add a legend and show the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

print("Plot created successfully!")